In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
from scripts.tools.issuetracker import IssuetrackerAPI
from os.path import join as pjoin
import re

In [2]:
def frs_nums(s):
    pos = s.find("FRS")
    num = s[pos+3:]
    return num.split(".")

def num_tuple(s):
    return tuple(int(i) for i in frs_nums(s))
    
def sort_key(t):
    return num_tuple(t[0])

def sort_frs_item(s):
    return num_tuple(s)

In [30]:
def paste_data(ws, data):
    cells = ws.Cells
    cr = cells.Range

    headers = [("FRS Number", "Level", "id_test", "Tested")]
    cr(cr("A1"), cr("A1").Offset(1, len(headers[0]))).Value = headers

    frs_start = cr("A2")
    frs_end = frs_start.Offset(len(data), 1)

    id_start = frs_start.Offset(1, 3)
    id_end = frs_end.Offset(1, 3)

    tested_start = id_start.Offset(1,2)
    tested_end = id_end.Offset(1,2)

    paste_start = frs_start
    paste_end = frs_end.Offset(1, len(data[0]))
    
    with screen_lock(ws.Application):
        paste_range = cr(paste_start, paste_end)
        print("Pasting test data")
        paste_range.Value = data

        print("Applying alignment formatting")
        cr(frs_start, frs_end).VerticalAlignment = xlc.xlTop
        cr(id_start, id_end).HorizontalAlignment = xlc.xlRight
        cr(tested_start, tested_end).HorizontalAlignment = xlc.xlRight
        cr(tested_start, tested_end).VerticalAlignment = xlc.xlTop

        print("Marking untested cells")
        col_os = paste_end.Column - paste_start.Column + 1
        for offset, (item, level, _, tested) in enumerate(data, 1):
            if not tested:
                row = cr(paste_start.Offset(offset, 1), paste_start.Offset(offset, col_os))
                row.Interior.Pattern = xlc.xlSolid
                row.Interior.PatternColorIndex = xlc.xlAutomatic
                row.Interior.ThemeColor = xlc.xlThemeColorAccent2
                row.Interior.TintAndShade = 0.399975585192419
                row.Interior.PatternTintAndShade = 0
                row[1].IndentLevel = level

        #print("Applying autofilter on untested items")
        #cr(paste_start.Offset(0, 1), paste_end).AutoFilter(Field=col_os, Criteria1="=")

        print("Applying column autofit")
        # fit after filter to account for width of filter icon
        for c in paste_range.Columns:
            c.EntireColumn.AutoFit()


def get_matrix_sheet(wb):
    matrix = None
    for ws in wb.Worksheets:
        if ws.Name == "Matrix":
            matrix = ws
            break
    if matrix is None:
        matrix = wb.Worksheets.Add()
        matrix.Name = "Matrix"
    # clear filters if present
    matrix.UsedRange.Clear()
    matrix.UsedRange.Clear()
    return matrix

In [31]:
_frs_match = re.compile(r"(.*FRS\d+)\.?([\d\.]*)").match
_toplevel_match = re.compile(r"^\>?[\+\*]{2}(FRS\d+)[\+\*]{2}.*$").match
_subitem_match = re.compile(r"^\>?[\+\*]+\s\*(FRS[\d\.]+)\:?\*\:?.*$").match

def dump(node, level=0):
    for k, v in sorted(node.children.items()):
        print(" "*level + str(v.id)+ " "+str(v._tests))
        dump(v, level+1)

class Node():
    def __init__(self, id, parent):
        self.id = id
        self.parent = parent
        self.children = {}
        self._tests = []
        
    def add_test(self, id_test):
        self._tests.append(id_test)
        for c in self.children.values():
            c.add_test(id_test)
        
    def get_tests(self):
        return sorted(self._tests)
        
    def get(self, id):
        return self.children.get(id, None)
        
    def add_child(self, id):
        child = self.mk_child(id)
        self.children[id] = child
        return child
    
    def mk_child(self, id):
        return self.__class__("%s.%s"%(self.id, id), self)
    
    def iter(self):
        # use .items() instead of .values() to sort
        for _, v in sorted(self.children.items()):
            yield v
            yield from v.iter()
            
    def __repr__(self):
        return "%s(%r)" % (self.__class__.__name__, self.id)
    
    def is_tested(self):
        if self._tests:
            return True
        elif not self.children:
            return False
        else:
            for c in self.children.values():
                if not c.is_tested():
                    return False
            return True
        
class Root(Node):
    def __init__(self):
        super().__init__("", None)
        
    def mk_child(self, id):
        return Node(id, self)
        
    def add(self, key):
        frs, nums = _frs_match(key).groups()
        child = self.get(frs)
        if not child:
            child = self.add_child(frs)
        if not nums:
            return child
        path = [int(i) for i in nums.split(".")]
        path.reverse()
        node = child
        while path:
            id = path.pop()
            child = node.get(id)
            if child is None:
                child = node.add_child(id)
            node = child
        return node
    
    def lookup(self, key):
        frs, nums = _frs_match(key).groups()
        node = self.get(frs)
        if not node or not nums:
            return node
        path = [int(i) for i in nums.split(".")][::-1]
        while path and node:
            id = path.pop()
            node = node.get(id)
        return node

In [32]:
KNOWN_WEBFRS_MAX = 121

class BadFRSNumber(Exception):
    pass

def load_user_test_map(ws):
    cells = ws.Cells
    cr = cells.Range
    id_start = cells.Find("ID_TEST").Offset(2, 1)
    id_end = id_start.End(xlc.xlDown)
    frs_start = cells.Find("List Web FRS").Offset(2, 1)
    frs_end = frs_start.Offset(id_end.Row - id_start.Row + 1, 1)
    test_data = cr(id_start, frs_end).Value
    test_map = []
    for row in test_data:
        id_test, frs = row[0], row[-1]
        iid_test = int(id_test)
        if id_test == iid_test:
            id_test = iid_test
        frs = frs or ""
        frs = [f.strip() for f in frs.split('\n')]
        for f in filter(None, frs):
            test_map.append((f, id_test))
        
    test_map.sort(key=sort_key)
    return test_map

def _build_tree(all_items):
    root = Root()
    for frs in all_items:
        root.add(frs)
    return root

def _add_tests(root, test_map):
    for frs, id_test in test_map:
        node = root.lookup(frs)
        if not node:
            raise BadFRSNumber(frs)
        node.add_test(id_test)

def make_frs_tree(test_map, all_items):
    root = _build_tree(all_items)
    _add_tests(root, test_map)
    return root
    
def load_frs_from_issuetracker():
    relevant = download_relevant_issues()
    all_frs = set()
    for v in relevant:
        lines = v.description.splitlines()
        for line in lines:
            for match in (_toplevel_match, _subitem_match):
                m = match(line)
                if m:
                    frs = m.group(1)
                    all_frs.add(frs)
    for i in range(1, KNOWN_WEBFRS_MAX + 1):
        all_frs.add("3.0WebFRS%03d" % i)
    return all_frs

def download_relevant_issues():
    api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    issues = api.download_issues("pbssoftware")
    relevant = []
    for v in issues.values():
        if v.sprint_milestone == "3.0" and \
            v.tracker == "Specification" and \
            v.status not in ("Closed", "Rejected"):
                relevant.append(v)
    return relevant

def make_paste_data(all_frs, root):
    paste_data = []
    for f in sorted(all_frs, key=sort_frs_item):
        node = root.lookup(f)
        if not node:
            tests = ()
            tested = ""
        else:
            tests = "\n".join(str(i) for i in node.get_tests())
            tested = "Y" if node.is_tested() else ""
        paste_data.append((f, f.count("."), tests, tested))
    return paste_data
        
def main(user_test_matrix, unit_test_matrix=None):
    xl = Excel()
    wb = xl.Workbooks.Open(user_test_matrix)
    ws = wb.Worksheets("Sheet1")
    
    user_test_map = load_user_test_map(ws)
    all_frs_items = load_frs_from_issuetracker()
    root_reqs = make_frs_tree(user_test_map, all_frs_items)
    data = make_paste_data(all_frs_items, root_reqs)
    
    ws2 = get_matrix_sheet(wb)
    paste_data(ws2, data)
    print("Done")
    

In [33]:
#def load_unit_test_map(unit_test_matrix):

In [45]:
trace_path = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\3.0 User Test\\Traceability"
matrix_file = "Tests 161111 2.xlsx"
old_matrix = "Traceability Matrix (Reqs and VI) with full FRS check 161109.xlsx"
p1 = os.path.join(trace_path, matrix_file)
p2 = os.path.join(trace_path, old_matrix)
main(p1, p2)

Pasting test data
Applying alignment formatting
Marking untested cells
Applying autofilter on untested items
Applying column autofit
Done


In [47]:
def fix_listwebfrs():
    xl = Excel()
    for c in xl.Selection:
        v = c.Value
        if not v: continue
        c.Value = v.replace("\n", "\r\n")
fix_listwebfrs()

In [51]:
def fix_indents():
    for c in xl.Selection:
        v = c.Value
        if v.startswith("3.0"): continue
        c.IndentLevel = v.count(".")
fix_indents()

In [39]:
tests = 0
passed = 0
errors = []
import pickle, os, datetime
from scripts.tools import issuetracker
_issues = None

def setup():
    global IssuetrackerAPI
    class MockAPI(issuetracker.IssuetrackerAPI):

        def download_issues(self, *args, _Force=False, **kw):
            cache = "issues_cache.pkl"
            issues = date = None
            reason = ""
            if os.path.exists(cache):
                with open(cache, 'rb') as f:
                    try:
                        date, issues = pickle.load(f)
                    except Exception as e:
                        reason = "Error: %s" % str(e)
            
            if not reason:
                if _Force:
                    reason = "Forced re-cache"
                elif issues is None:
                    reason = "No existing cache found."
                elif date < datetime.date.today():
                    reason = "Old cache"
            
            if reason:
                print("Redownloading issues:", reason)
                issues = super().download_issues(*args, **kw)
                global _issues
                _issues = issues
                self._cache_issues(cache, issues)
            return issues

        def _cache_issues(self, cache, issues):
            date = datetime.date.today()
            with open(cache, 'wb') as f:
                ob = (date, issues)
                pickle.dump(ob, f)
    IssuetrackerAPI = MockAPI
setup()
            
def fail(msg):
    global tests, errors
    tests += 1
    errors.append(msg)

def success():
    global tests, passed
    tests += 1
    passed += 1
    
def assert_equal(a, b, func=None):
    if a != b:
        if func:
            msg = func(a, b)
        else:
            msg = "%r != %r" % (a, b)
        fail(msg)
    else:
        success()
        
def get_root():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    return root
        
###########
# Tests 
###########
def test_root_all_frs():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    seen = {v.id for v in root.iter()}
    assert_equal(seen, all_frs)
test_root_all_frs()

def test_root_lookup():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    ids = {v.id for v in root.iter()}
    assert_equal(ids, all_frs)
    s1 = []
    s2 = []
    for id in ids:
        s1.append(root.lookup(id).id)
        s2.append(id)
    assert_equal(s1, s2)
test_root_lookup()

def test_frs_strings():
    xl = Excel()
    wb = xl.Workbooks.open(test_file_path)
    ws = wb.Worksheets("Sheet1")
    test_map = load_test_map(ws)
    all_frs = load_frs_from_issuetracker
    frs_strings = {f for f, _ in test_map}
    diff = frs_strings - all_frs
    bad_tests = {}
    for d in diff:
        tests = test_map2.get(d)
        for t in tests:
            bad = bad_tests.get(t, None)
            if bad is None:
                bad_tests[t] = bad = []
            bad.append(d)
    #     print(test_map2.get(d), d)
    for t, frs in sorted(bad_tests.items()):
        print(t, frs)
    if not bad_tests:
        print("No Bad Tests Found")
    else:
        assert False, "Bad Tests found"
    
###########
# Cleanup 
###########
    
def teardown():
    global IssuetrackerAPI
    IssuetrackerAPI = issuetracker.IssuetrackerAPI
    
def finish():
    teardown()
    print("%d / %d tests passed" % (passed, tests))
    if errors:
        print("Errors found")
        for e in errors:
            print(e)
    else:
        print("Success")
finish()

Redownloading issues Old cache
3 / 3 tests passed
Success
